In [ ]:
# this is just a temp fix to get things working locally. use the server config when starting jupyter
import sys
sys.path.append('~/development/websocket_with_c_driver/client_side')

In [ ]:
from wsclient.client import TestWebSocketClient
from tornado import ioloop

client = TestWebSocketClient()
client.connect('ws://127.0.0.1:8675/ws')

# i guess an ioloop already exists and we don't need this? from the client_main.py file.
#    try:
#        ioloop.IOLoop.instance().start()
#    except KeyboardInterrupt:
#        client.close()


In [ ]:
d = {'does_it': 'blend'}
client.send(d)